In [2]:
import pyodbc
import pandas as pd
import plotly.express as px

# Connect to SQL Server ---
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=.;'
    'DATABASE=AdventureWorks2019;'
    'Trusted_Connection=yes;'
)

#  Call the stored procedure ---
df = pd.read_sql_query("EXEC usp_GetCountrySales", conn)

#  Inspect DataFrame ---
print(df.head())


C:\Users\mayan\AppData\Local\Temp\ipykernel_25096\1109885633.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("EXEC usp_GetCountrySales", conn)


      Region  RegionalSales
0  Southwest    24184609.60
1  Northwest    16084942.55
2    Central     7909009.01
3  Southeast     7879655.07
4  Northeast     6939374.48


In [13]:
import plotly.express as px
import plotly.graph_objects as go

region_states = {
    "Northwest": ['AK', 'HI','ID','MT','NV','OR','UT','WA','WY'],
    "Southwest": ['AZ', 'CA', 'GU', 'NM', 'TX' ],
    "Central":   [ 'AR','CO','IA','IL','KS','LA','MI','MN', 'MO', 'ND', 'NE', 'OK', 'SD', 'WI'],
    "Northeast": ['CT', 'DC','DE','IN','MA','MD','ME','NH','NJ','NY','OH','PA', 'RI','VT'],
    "Southeast": ['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'PR', 'SC', 'TN', 'VA', 'WV']
}

df['Percent'] = df['RegionalSales'] / df['RegionalSales'].sum() * 100

#  Expand DataFrame to State Level ---
rows = []
for region, states in region_states.items():
    regional_sales = df.loc[df['Region'] == region, 'RegionalSales'].values[0]
    percent = df.loc[df['Region'] == region, 'Percent'].values[0]
    for state in states:
        rows.append([state, region, regional_sales, percent])

df_states = pd.DataFrame(rows, columns=['State', 'Region', 'RegionalSales', 'Percent'])

# Plot Choropleth ---
fig = px.choropleth(
    df_states,
    locations='State',
    locationmode='USA-states',
    color='Region',
    scope='usa',
    hover_name='Region',
    hover_data={
        'RegionalSales': True,
        'Percent': ':.2f',
        'State': False
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Add RegionalSales Labels at Region Centroids ---
region_coords = {
    "Northwest": [46.5, -115],
    "Southwest": [34, -112],
    "Central":   [43, -95],
    "Northeast": [42, -74],
    "Southeast": [31.5, -82]
}

for _, row in df.iterrows():
    region = row['Region']
    lat, lon = region_coords[region]
    
    sales_text = f"${row['RegionalSales']:,.0f}"
    pct_text = f"{row['Percent']:.1f}%"
    region_label = f"<b>{region}</b><br>{sales_text}<br>{pct_text}"


    fig.add_trace(go.Scattergeo(
       lon=[lon],
        lat=[lat],
        text=[region_label],
        mode='text',
        textfont=dict(size=14, color='black'),
        showlegend=False   # <<< hides text traces from legend
    ))


#  Layout ---
fig.update_layout(
    title_text='Regional Sales of best performing Country(US)',
    title_x=0.5,
    geo=dict(showland=True, landcolor='lightgray'),
    legend=dict(
        x=0.80,    # move legend horizontally (0 = left, 1 = right)
        y=0.98,    # move legend vertically   (1 = top)
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="gray",
        borderwidth=1
    )
)

fig.show()

In [3]:
# --- Step 5: Close connection ---
conn.close()